<a href="https://colab.research.google.com/github/erinijapranckeviciene/MLdata/blob/main/Saliu_panasumai_isuic_regresijos_koeficientu_klasterizavimas_EP_VDU_2021_DG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [44]:
import pandas as pd
# ikeliu duomenis is Github, taip patogiau
url="https://raw.githubusercontent.com/erinijapranckeviciene/MLdata/main/Kaggle-data-suic.csv"
data = pd.read_csv(url)

# Panaudosim klasterizavimą ištirti šalių panašumą pagal isuic tendencijas- kitimą pagal metus atskirai vyrams ir moterims 

In [45]:
print(data.dtypes)

country                object
year                    int64
sex                    object
age                    object
suicides_no             int64
population              int64
suicides/100k pop     float64
country-year           object
HDI for year          float64
 gdp_for_year ($)      object
gdp_per_capita ($)      int64
generation             object
dtype: object


# Mums reikia pasilikti tik kelis kintamuosius kol kas ir pasidaryti šalių vektorių kad galėtume jį panaudoti cikle

In [46]:
data1=data[['country','year','suicides/100k pop','sex','age']].copy()
data1.rename(columns={'suicides/100k pop':'suicides'}, inplace=True)
country=data['country'].unique()
data1.describe(include='all')


,country,year,suicides,sex,age
count,27820,27820.000000,27820.000000,27820,27820
unique,101,NaN,NaN,2,6
top,Iceland,NaN,NaN,male,15-24 years
freq,382,NaN,NaN,13910,4642
mean,NaN,2001.258375,12.816097,NaN,NaN
std,NaN,8.469055,18.961511,NaN,NaN
min,NaN,1985.000000,0.000000,NaN,NaN
25%,NaN,1995.000000,0.920000,NaN,NaN
50%,NaN,2002.000000,5.990000,NaN,NaN
75%,NaN,2008.000000,16.620000,NaN,NaN


In [47]:
data1.head()

,country,year,suicides,sex,age
0,Albania,1987,6.71,male,15-24 years
1,Albania,1987,5.19,male,35-54 years
2,Albania,1987,4.83,female,15-24 years
3,Albania,1987,4.59,male,75+ years
4,Albania,1987,3.28,male,25-34 years


Galim bandyti seaborn įvairiais atvaizdavimo būdais, bet dabar nebandysim
https://seaborn.pydata.org/examples/horizontal_boxplot.html 

In [ ]:
import seaborn as sns
#sns.pairplot(data1, hue="sex" )

# Kad sudaryti regresijos modelius, kiekvienos šalies duomenis reikės atskirai susitvarkyti. Šituose duomenyse suicides taškas yra skirtingas sex ir age, reiškia reikės pakeisti kategorinius į 'one-hot' kodavimą. Pakeitimus atliekam cikle kiekvienai šaliai. Galim ir iš kart skaičiuoti regresijos koeficientus ir formuoti požymių masyvą. 

In [48]:
#pasitikrinam kad viskas gaunasi ir atskiriam šalis
salysp = []

for c in country:
  cdata=data1[(data1['country']==c)]
  #print(type(cdata))
  #print(cdata.head())
  row, col = cdata.shape
  print(c,row)
  salysp.append([c, row])

salys = pd.DataFrame(salysp, columns=['salis', 'n'])
salys.head()

Albania 264
Antigua and Barbuda 324
Argentina 372
Armenia 298
Aruba 168
Australia 360
Austria 382
Azerbaijan 192
Bahamas 276
Bahrain 252
Barbados 300
Belarus 252
Belgium 372
Belize 336
Bosnia and Herzegovina 24
Brazil 372
Bulgaria 360
Cabo Verde 12
Canada 348
Chile 372
Colombia 372
Costa Rica 360
Croatia 262
Cuba 288
Cyprus 178
Czech Republic 322
Denmark 264
Dominica 12
Ecuador 372
El Salvador 288
Estonia 252
Fiji 132
Finland 348
France 360
Georgia 264
Germany 312
Greece 372
Grenada 310
Guatemala 360
Guyana 300
Hungary 310
Iceland 382
Ireland 360
Israel 372
Italy 372
Jamaica 204
Japan 372
Kazakhstan 312
Kiribati 132
Kuwait 300
Kyrgyzstan 312
Latvia 252
Lithuania 262
Luxembourg 372
Macau 12
Maldives 120
Malta 372
Mauritius 382
Mexico 372
Mongolia 10
Montenegro 120
Netherlands 382
New Zealand 348
Nicaragua 72
Norway 360
Oman 36
Panama 300
Paraguay 324
Philippines 180
Poland 288
Portugal 324
Puerto Rico 372
Qatar 178
Republic of Korea 372
Romania 334
Russian Federation 324
Saint Kitts and

,salis,n
0,Albania,264
1,Antigua and Barbuda,324
2,Argentina,372
3,Armenia,298
4,Aruba,168


#Atliekant patikrinimą galim pastebėti kad šalys gali skirtis pagal tai kad kai kuriems metams nėra duomenų. Kol kas pasiliekam taip, o paskui pasirinksim tik tas kurioms yra atitikimas pagal visus metus. 

# Susikonstruojam regresijos modelius kuriuos nagrinėsim

In [49]:
from sklearn.metrics import explained_variance_score
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score

from time import time

from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import LinearRegression, RANSACRegressor, Lasso, Ridge
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.svm import SVR
from sklearn.svm import LinearSVR

regressors = [
    LinearRegression(), 
    #RANSACRegressor(), 
    KNeighborsRegressor(),
    KNeighborsRegressor(n_neighbors=9, metric='manhattan'),
    SVR(),
    #LinearSVR(),
    Lasso(),
    Ridge()
    #GaussianProcessRegressor(),
]


# Ištiriam regresijos modelius kurių koeficientų duomenis naudoti klasterizavimui

In [ ]:
rez = []
# Atliekam modelių tyrimą, rezultatus susirenkam į dataframe
for c in country:
  print(c)
  #paliekam tik šalies duomenis 
  cdata=data1[(data1['country']==c)]
  y_train=cdata['suicides']
  #išmetam šalies kintamąjį ir taget kintamąjį
  cdata1=cdata.drop(columns=['country','suicides'],axis=1)
  X_train=pd.get_dummies(cdata1)
  for model in regressors:
    start = time()
    model.fit(X_train, y_train)
    train_time = time() - start
    start = time()
    predictions = model.predict(X_train)
    predict_time = time()-start    
    #print(model)
    #print("\tTraining time: %0.3fs" % train_time)
    #print("\tPrediction time: %0.3fs" % predict_time)
    #print("\tExplained variance:", explained_variance_score(y_train, predictions))
    evc=explained_variance_score(y_train, predictions)
    #print("\tMean absolute error:", mean_absolute_error(y_train, predictions))
    mae=mean_absolute_error(y_train, predictions)
    #print("\tR2 score:", r2_score(y_train, predictions))
    r2=r2_score(y_train, predictions)
    #print()
    rez.append([c, model, evc, mae, r2, train_time])

rezdf = pd.DataFrame(rez, columns=['country', 'model', 'explainedVS','MeanAbsErr','R2','Time'])
rezdf.head()

# Išanalizuojam rezultatų matricą ir pasirenkam modelį kuris geriausias ar keli geriausi

In [42]:
rezdf
# geriau įsirašyti į google drve tokią didelę lentelę
from google.colab import drive
drive.mount('drive')

Mounted at drive


In [43]:
rezdf.to_csv('regrrez.csv')
!cp regrrez.csv "drive/My Drive/"

# Rezultatai rodo kad geriausiai daugymai šalių veikia nerest neighbor regression, bet kaip pasiimti koeficientus?  Ir Ridge regression . Tad pasirinkime Ridge. 

In [ ]:
#atsargines eilutes
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

In [ ]:
r2_score?

Pastebėjimas Regresijos modelis Ridge "užnulina" nereikšmingų požymių koeficientus

In [89]:
from sklearn.linear_model import Lasso, Ridge
model = Ridge()

rez = []
# Atliekam modelių tyrimą, rezultatus susirenkam į dataframe
for c in country:
  #print(c)
  #paliekam tik šalies duomenis 
  cdata=data1[(data1['country']==c)]
  y_train=cdata['suicides']
  #išmetam šalies kintamąjį ir taget kintamąjį
  cdata1=cdata.drop(columns=['country','suicides'],axis=1)
  X_train=pd.get_dummies(cdata1)
  model.fit(X_train, y_train)
  predictions = model.predict(X_train)
  evc=explained_variance_score(y_train, predictions)
  mae=mean_absolute_error(y_train, predictions)
  r2=r2_score(y_train, predictions)
  # prijunkime ir regresijos koeficientus ir regresijos metrikas
  rez1=model.coef_
  print(c, rez1[7], r2, len(rez1))
  if len(rez1)==9:
   rez.append([c, evc, mae, r2, model.intercept_, model.coef_[0], model.coef_[1], model.coef_[2], model.coef_[3], model.coef_[4], model.coef_[5],model.coef_[6], model.coef_[7], model.coef_[8] ] )
#columns=['country', 'explainedVS','MeanAbsErr','R2','intercept','year', 'sex_female', 'sex_male', 'age_15-24_years', 'age_25-34_years','age_35-54_years', 'age_5-14_years', 'age_55-74_years','age_75+ years'])

Albania -0.3990370370370342 0.2956039373875816 9
Antigua and Barbuda 1.288606060606072 0.0646821226732901 9
Argentina 2.720820105820031 0.6748494080056677 9
Armenia 1.6620440921853166 0.6220446496521241 9
Aruba 1.095287356321845 0.15903384107408747 9
Australia 0.4923224043715504 0.8178205923609095 9
Austria 6.067848288397195 0.7754958630408019 9
Azerbaijan 0.8035353535353504 0.39642092442142796 9
Bahamas -0.23340425531913186 0.14322765153845085 9
Bahrain -0.4112403100775102 0.3631426773850909 9
Barbados 1.180947712418212 0.2548592955680484 9
Belarus 13.172131782945769 0.7754386631001102 9
Belgium 5.2823809523808976 0.7558764458035128 9
Belize 0.29429824561405404 0.24339313333712398 9
Bosnia and Herzegovina 1.6043333333333365 0.5482846642913349 9
Brazil 1.3591534391533775 0.7715381012570159 9
Bulgaria 4.60530054644808 0.7702016626410184 9
Cabo Verde -1.8355555555555554 0.6243351123779382 9
Canada 1.7481355932202722 0.8457299015569091 9
Chile 2.352116402116403 0.7601816152509072 9
Colomb

# Pasidarom rezultatų lentelę klasterizavimui. Taip pat pagalvokim kaip vizualizuoti regresijos modelius kuriuos pasigaminom, kurie prognozuoja savižudybių skaičiaus kitimą šalyse pagal metus

In [91]:
#names=X_train.columns
#print(names)
rezdf = pd.DataFrame(rez, columns=['country', 'explainedVS','MeanAbsErr','R2','intercept','year', 'sex_female', 'sex_male', 'age_15-24 years', 'age_25-34 years','age_35-54 years', 'age_5-14 years', 'age_55-74 years','age_75+ years'])
rezdf.head()


,country,explainedVS,MeanAbsErr,R2,intercept,year,sex_female,sex_male,age_15-24 years,age_25-34 years,age_35-54 years,age_5-14 years,age_55-74 years,age_75+ years
0,Albania,0.295604,2.165933,0.295604,-33.645980,0.018582,-1.078195,1.078195,0.987852,0.755407,0.135185,-3.000815,-0.399037,1.521407
1,Antigua and Barbuda,0.064682,1.249164,0.064682,-25.845378,0.013203,-0.453681,0.453681,-0.542848,-0.270121,0.610061,-0.542848,1.288606,-0.542848
2,Argentina,0.674849,4.992420,0.674849,169.092358,-0.079312,-6.637674,6.637674,-1.595847,-2.637275,-1.907116,-9.631402,2.720820,13.050820
3,Armenia,0.622045,1.563553,0.622045,48.193018,-0.022437,-1.657967,1.657967,-1.966191,-1.056191,0.014005,-3.007083,1.662044,4.353417
4,Aruba,0.159034,12.804277,0.159034,2091.725540,-1.038885,-6.140118,6.140118,-5.251954,3.460115,0.167011,-9.175402,1.095287,9.704943


# Kaip vizualizuoti , ką vzualizuoti, ką galėtume ištirti šitoj lentelėj?  Čia mūsų duomenys klasterizavimo algoritmų taikymui. 